<a href="https://colab.research.google.com/github/wiendewani/Praktikum_ML/blob/putri/%5BPretrained%5D_Preprocessing_%2B_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Download Data

gak perlu di run lagi

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
! pip install kaggle

In [ ]:
! mkdir ~/.kaggle

In [ ]:
! cp kaggle.json ~/.kaggle/

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d vitaminc/cigarette-smoker-detection

100% 2.50G/2.50G [00:34<00:00, 50.7MB/s]
100% 2.50G/2.50G [00:34<00:00, 78.2MB/s]


In [ ]:
import os
import zipfile

In [ ]:
local_zip = '/content/cigarette-smoker-detection.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/content/drive/MyDrive/Pembelajaran Mesin/Project ML')
zip_ref.close()

In [ ]:
!pip install split-folders

In [ ]:
!mkdir "/content/drive/MyDrive/Pembelajaran Mesin/Project ML/dataset"

In [ ]:
import splitfolders

splitfolders.ratio("/content/drive/MyDrive/Pembelajaran Mesin/Project ML/data/data", output="/content/drive/MyDrive/Pembelajaran Mesin/Project ML/dataset", seed= 1337, ratio =(.8,.2), group_prefix=None)

Copying files: 3275 files [01:39, 32.80 files/s]


### Import Library

In [ ]:
!pip install split-folders

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from imutils import paths
import matplotlib.pyplot as plt
import os
import cv2
import numpy as np
import splitfolders

### Load Data

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
splitfolders.ratio("/content/drive/MyDrive/Pembelajaran Mesin/Project ML/dataset", 
                   output="/content/drive/MyDrive/Praktikum Semester 7/Modul 4/splitting", 
                   seed= 1337, 
                   ratio =(.8,.19,.01), 
                   group_prefix=None)

## Modelling

#### pretrained plus cnn

In [ ]:
base_dir = 'D:\Wien\Project_ML\Dataset'

train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'val')
test_dir = os.path.join(base_dir, 'test')

train_Not_Smoking_dir = os.path.join(train_dir, 'not_smoking')
train_Smoking_dir = os.path.join(train_dir, 'smoking')

validation_Not_Smoking_dir = os.path.join(validation_dir, 'not_smoking')
validation_Smoking_dir = os.path.join(validation_dir, 'smoking')

test_Not_Smoking_dir = os.path.join(test_dir, 'not_smoking')
test_Smoking_dir = os.path.join(test_dir, 'smoking')

In [ ]:
import cv2
import numpy as np


# 2.1 Gather data train & test
train_data = []
train_label = []
for r, d, f in os.walk(train_dir):
    for file in f:
        if ".png" in file:
            imagePath = os.path.join(r, file)
            image = cv2.imread(imagePath)
            image = cv2.resize(image, (150,150))
            train_data.append(image)
            label = imagePath.split(os.path.sep)[-2]
            train_label.append(label)

train_data = np.array(train_data)
train_label = np.array(train_label)

val_data = []
val_label = []
for r, d, f in os.walk(validation_dir):
    for file in f:
        if ".png" in file:
            imagePath = os.path.join(r, file)
            image = cv2.imread(imagePath)
            image = cv2.resize(image, (150,150))
            val_data.append(image)
            label = imagePath.split(os.path.sep)[-2]
            val_label.append(label)

val_data = np.array(val_data)
val_label = np.array(val_label)

test_data = []
test_label = []
for r, d, f in os.walk(test_dir):
    for file in f:
        if ".png" in file:
            imagePath = os.path.join(r, file)
            image = cv2.imread(imagePath)
            image = cv2.resize(image, (150,150))
            test_data.append(image)
            label = imagePath.split(os.path.sep)[-2]
            test_label.append(label)

test_data = np.array(test_data)
test_label = np.array(test_label)


# Data shape
print("Train Data = ", train_data.shape)
print("Train Label = ", train_label.shape)
print("Val Data = ", val_data.shape)
print("Val Label = ", val_label.shape)
print("Test Data = ", test_data.shape)
print("Test Label = ", test_label.shape)

In [ ]:
# 2.2 Normalization

print("Before ", train_data[0][0][0])

x_train = train_data.astype('float32') / 255.0
x_test = test_data.astype('float32') / 255.0
x_val = val_data.astype('float32') / 255.0

print("After ", x_train[0][0][0])

In [ ]:
# 2.3 Label encoder
from sklearn.preprocessing import LabelEncoder

print("Before ", train_label[5:])

lb = LabelEncoder()
y_train = lb.fit_transform(train_label)
y_test = lb.fit_transform(test_label)
y_val = lb.fit_transform(val_label)

print("After ", y_train[5:])

In [ ]:
import tensorflow as tf

class myCallbacks(tf.keras.callbacks.Callback):
    def on_epoch_end(self,epoch,logs={}):
        if (logs.get('val_acc') >= 0.85):
            print("\nReached 85% accuracy so cancelling training!")
            self.model.stop_training=True

callback = myCallbacks()

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
aug = ImageDataGenerator(rotation_range=30, width_shift_range=0.1,
                        height_shift_range=0.1, shear_range=0.2, 
                        zoom_range=0.8, horizontal_flip=True,
                        fill_mode="nearest")

In [ ]:
# Pertama, kita import dulu library yang dibutuhkan
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Dropout, Input
from tensorflow.keras.applications.vgg16 import VGG16

import ssl

ssl._create_default_https_context = ssl._create_unverified_context

# Kita load model VGG16, kemudian kita potong bagian Top atau Fully Connected Layernya
baseModel = VGG16(include_top=False, input_tensor=Input(shape=(150, 150, 3)))

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import InputLayer, Dense, Conv2D, MaxPool2D, Flatten, Dropout

class FCHeadNet:
  def build(baseModel, classes, D):
    headModel = baseModel.output
#     headModel = Flatten(name="flatten")(headModel)
    headModel = Conv2D(filters=16, kernel_size=3, strides=1, padding='same', activation='elu')(headModel)
    headModel = BatchNormalization()(headModel)
    headModel = MaxPool2D(pool_size=2, padding='same')(headModel)
    headModel = Conv2D(filters=32, kernel_size=3, strides=1, padding='same', activation='elu')(headModel)
    headModel = BatchNormalization()(headModel)
    headModel = MaxPool2D(pool_size=2, padding='same')(headModel)
    headModel = Conv2D(filters=64, kernel_size=3, strides=1, padding='same', activation='elu')(headModel)
    headModel = BatchNormalization()(headModel)
    headModel = MaxPool2D(pool_size=2, padding='same')(headModel)
    headModel = Dropout(0.25)(headModel)
    headModel = Flatten()(headModel)
    headModel = Dense(128, activation='elu')(headModel)
    headModel = Dropout(0.5)(headModel)
    headModel = Dense(1, activation='sigmoid')(headModel)
    return headModel

headModel = FCHeadNet.build(baseModel, 1, 256)
model = Model(inputs=baseModel.input, outputs=headModel)

model.summary()

In [ ]:
from tensorflow.keras.optimizers import Adam

model.compile(optimizer=Adam(lr=1e-3), 
              loss='binary_crossentropy', 
              metrics=['acc'])

H = model.fit(aug.flow(x_train, y_train, batch_size=20), validation_data=(x_val, y_val), epochs=100, callbacks=callback)

In [ ]:
import matplotlib.pyplot as plt

plt.rcParams['axes.spines.right'] = False
plt.rcParams['axes.spines.top'] = False

def plot_graphs_acc(history, title='Accuracy'):
  plt.figure(figsize=(12, 7))
  plt.title(title, fontsize=16)
  plt.plot(history.history['acc'])
  plt.plot(history.history['val_acc'])
  plt.xlabel("Epochs")
  plt.ylabel("Accuracy")
  plt.yticks(np.arange(0, 1, step=0.05))
  plt.legend(['Training data', 'Test data'])
  plt.show()

def plot_graphs_loss(history, title='Loss'):
  plt.figure(figsize=(12,7))
  plt.title(title, fontsize=16)
  plt.plot(history.history['loss'])
  plt.plot(history.history['val_loss'])
  plt.xlabel("Epochs")
  plt.ylabel("Loss")
  plt.yticks(np.arange(0, 4, step=0.5))
  plt.legend(['Training data', 'Test data'])
  plt.show()

# Model
plot_graphs_acc(H)
print("\n")
plot_graphs_loss(H)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

pred = model.predict(x_test)
labels = (pred > 0.5).astype(np.int)

print(classification_report(y_test, labels))

In [ ]:
results = confusion_matrix(np.array(y_test.argmax(axis=1)), labels.argmax(axis=1))
fig, ax = plt.subplots(figsize=(7.5, 7.5))
ax.matshow(results, cmap=plt.cm.Blues, alpha=0.9)
for i in range(results.shape[0]):
    for j in range(results.shape[1]):
        ax.text(x=j, y=i,s=results[i, j], va='center', ha='center', size='xx-large')
 
plt.xlabel('Prediction', fontsize=14)
plt.ylabel('Actual Label', fontsize=14)
plt.title('Confusion Matrix', fontsize=18)
plt.show()

#### pretrained tapi dua

In [ ]:
# Pertama, kita import dulu library yang dibutuhkan
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Dropout, Input
from tensorflow.keras.applications.xception import Xception

import ssl

ssl._create_default_https_context = ssl._create_unverified_context

baseModel = VGG16(include_top=False, input_tensor=Input(shape=(150, 150, 3)))
baseModel2 = Xception(include_top=False, input_tensor=Input(shape=(150, 150, 3)))

In [ ]:
IMAGE_SIZE = (150,150,3)

vgg19 = tf.keras.applications.vgg19.VGG19(
    input_shape=IMAGE_SIZE, weights='imagenet', include_top=False)
for layer in vgg19.layers:
    layer._name = layer._name + str('_19')
    layer.trainable = False

vgg16 = tf.keras.applications.vgg16.VGG16(
    input_shape=IMAGE_SIZE, weights='imagenet', include_top=False)
for layer in vgg16.layers:
    layer._name = layer._name + str('_16')
    layer.trainable = False

inp = Input(IMAGE_SIZE)
    
vgg16_x = Flatten()(vgg16(inp))
vgg19_x = Flatten()(vgg19(inp))

x = Concatenate()([vgg16_x, vgg19_x])
out = Dense(1, activation='sigmoid')(x)

model = Model(inputs = inp, outputs = out)

model.summary()

In [ ]:
model.compile(optimizer=Adam(lr=1e-3), 
              loss='binary_crossentropy', 
              metrics=['acc'])

H2 = model.fit(aug.flow(X_train, y_train, batch_size=20), validation_data=(X_val, y_val), epochs=100, callbacks=callback)

In [ ]:
plot_graphs_acc(H2)
print("\n")
plot_graphs_loss(H2)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

pred = model.predict(x_test)
labels = (pred > 0.5).astype(np.int)

print(classification_report(y_test, labels))

In [ ]:
results = confusion_matrix(np.array(y_test.argmax(axis=1)), labels.argmax(axis=1))
fig, ax = plt.subplots(figsize=(7.5, 7.5))
ax.matshow(results, cmap=plt.cm.Blues, alpha=0.9)
for i in range(results.shape[0]):
    for j in range(results.shape[1]):
        ax.text(x=j, y=i,s=results[i, j], va='center', ha='center', size='xx-large')
 
plt.xlabel('Prediction', fontsize=14)
plt.ylabel('Actual Label', fontsize=14)
plt.title('Confusion Matrix', fontsize=18)
plt.show()